# Data Definition Language - DDL
Let us create a database `movielens`

In [ ]:
%load_ext sql
%sql hive://hadoop@localhost:10000/

Note, to delete the entire database

```sql
%%sql
DROP DATABASE IF EXISTS movielens CASCADE
```

In [ ]:
%%sql
CREATE DATABASE IF NOT EXISTS movielens

In [ ]:
%sql SHOW DATABASES

In [ ]:
%sql USE MOVIELENS

## Defining the Tables and Loading the Data

### Movies

Note, to drop the table `movies`:
```sql
%SQL DROP TABLE movies
```

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS movies (
    movieId int,
    title String,
    year int,
    genres ARRAY<STRING>
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
COLLECTION ITEMS TERMINATED BY "|"
LINES TERMINATED BY '\n'
STORED AS TEXTFILE
tblproperties ("skip.header.line.count"="1")

In [ ]:
%%sql

LOAD DATA LOCAL INPATH '/data/dataset/movielens/ml-25m/movies_cleaned.csv'
OVERWRITE INTO TABLE movies

## Ratings

In [ ]:
%%sql

CREATE TABLE IF NOT EXISTS ratings (
    userid INT, 
    movieid INT,
    rating INT, 
    `timestamp` bigint
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
STORED AS TEXTFILE
tblproperties ("skip.header.line.count"="1")

In [ ]:
%%sql

LOAD DATA LOCAL INPATH '/data/dataset/movielens/ml-25m/ratings.csv'
OVERWRITE INTO TABLE ratings

## Creating a Joined Table

In [ ]:
%%sql
CREATE TABLE movie_rating
AS SELECT 
    m.*, 
    r.num_rating, 
    r.first_quartile_rating,
    r.median_rating,
    r.third_quartile_rating,
    r.avg_rating,
    r.min_rating,
    r.max_rating FROM (
    SELECT
        movieid,
        count(rating) as num_rating,
        percentile(rating, 0.25) as first_quartile_rating,
        percentile(rating, 0.5) as median_rating,
        percentile(rating, 0.75) as third_quartile_rating,
        avg(rating) as avg_rating,
        min(rating) as min_rating,
        max(rating) as max_rating
    FROM ratings
    GROUP BY movieid
) r
RIGHT JOIN movies m ON m.movieid = r.movieid

## Simple Checks

In [ ]:
%sql select * from movies limit 2

In [ ]:
%sql select * from ratings limit 2

In [ ]:
%sql select * from movie_rating limit 2

In [ ]:
%sql select count(*) from movies

In [ ]:
%sql select count(*) from ratings

In [ ]:
%sql select count(*) from movie_rating